In [ ]:
import matplotlib.pyplot as plt
import polars as pl

from monkey import get_df, find_latest_csv, to_int, to_struct


# pathname = 'C:/Users/maple/Downloads/results*.csv'
pathname = '/home/chrlz/Downloads/results*.csv'

csv = find_latest_csv(pathname)
print(csv)


pl.Config.set_tbl_hide_column_data_types(True)


df = get_df(csv).with_columns(
    pl.col('charStats').str.split(',').list.eval(
        pl.element().cast(pl.UInt16)).list.to_struct(),
    to_struct('_id').name.keep(),
    # pl.col('_id').alias('original_id'),
)

def print_stats(df: pl.DataFrame):
    mean = df['wpm'].mean()
    restarts = df['restartCount'].sum()/len(df)
    print(f'Mean: {mean}')
    print(f'Restarts: {restarts}')
    print('Worst:')
    display(df.sort('wpm').head(15).sort(
        'index', descending=True).select('wpm', 'index').transpose())
    print('Earliest:')
    display(df.head(15).select('wpm', 'index').transpose())
    print('Best:')
    display(df.sort('wpm', descending=True).head(20))

# plt.hist(df.select('wpm'), bins=40)
# print_stats(df)
# for d in df:
#     print(d.name, len(d.unique()))


pl.Config.set_tbl_hide_column_data_types(False)
print(len(open(csv, 'rb').read()))
print(df.estimated_size())
pl.Config.set_fmt_str_lengths(10000)
df.with_columns(to_int('_id').alias('reconstructed')).filter(pl.col('isPb'))

print_stats(df)

In [ ]:
import polars as pl
from monkey import format_as_monkey, get_all


pathname = '/home/chrlz/Downloads/results*.csv'
get_all(pathname).pipe(format_as_monkey).write_csv('/home/chrlz/Downloads/results(34).csv', separator='|')

# pl.read_csv( '/home/chrlz/Downloads/results(1).csv', separator='|')

In [ ]:
import polars as pl
from monkey import DTYPES, add_index, format_df

df = pl.read_parquet('./results.parquet').with_columns([
    pl.col(x).cast(t)
    for x,t in DTYPES.items()
]).pipe(format_df).sort('index', descending=True).pipe(add_index)

# pl.read_csv('./results.tsv', separator='\t')

In [ ]:
import monkey as mk
# df.tail(len(df)-1000).sort('wpm', descending=True)
# df.head(1000).sort('wpm').head(20).sort('index')


def split_char_stats(df: pl.DataFrame):
    col = pl.col('charStats')
    return (
        df.with_columns( col.str.split(','))
        .with_columns(col.list.eval(pl.element().cast(pl.UInt16)))
        .with_columns([
            col.list.get(i).alias(name)
            for i, name in enumerate(['correct', 'incorrect', 'extra', 'missed'])
        ])
        .drop('charStats')
    )

neg_quote_len = (
pl.when(pl.col('quoteLength').is_null())
    .then(pl.lit(-1))
    .otherwise(pl.col('quoteLength'))
    .name.keep()
)

false_is_pb = (

    pl.when(pl.col('isPb') == 'true')
    .then(pl.lit(True))
    .otherwise(pl.lit(False))
    .name.keep()
)


df.pipe(mk.format_as_monkey).with_columns(
    false_is_pb,
    neg_quote_len
).pipe(split_char_stats).write_parquet('./results-2.parquet', compression='zstd')

In [ ]:
import polars as pl
df2= pl.read_parquet('./results.parquet')

In [ ]:
from monkey import format_df
# df2
df2.tail(1000-len(df2)).sort('timestamp', descending=True).pipe(format_df).sort('wpm').select('wpm', 'index')[0].to_dicts()[0]

In [ ]:
a=["ho","lo"]
b=["pe","ve"]
print(" ".join("".join(x) for x in zip(a,b)))